In [1]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:04<00:00,  1.21it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [ ]:

from PIL import Image

# Load images and pair them with placeholders
img_paths = ["path_to_image1.jpg", "path_to_image2.jpg"]
placeholders = [f"<image{i+1}>" for i in range(len(img_paths))]
image_placeholder_pairs = [(Image.open(img_path), placeholder) for img_path, placeholder in zip(img_paths, placeholders)]

# Create a structured prompt with placeholders
prompt = "Analyze the relationship between the images."
formatted_prompt = " ".join(placeholder for _, placeholder in image_placeholder_pairs) + " " + prompt

# Construct messages for the processor
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img},  # Attach image explicitly
            {"type": "text", "text": f"{placeholder}"}
        ]
    }
    for img, placeholder in image_placeholder_pairs
]


In [5]:


url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/0052a70beed5bf71b92610a43a52df6d286cd5f3/diffusers/rabbit.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = Image.open("/home/ubuntu/project/002877-R1-008-2A.jpg")

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": "Tell me what did you see in this picture?"}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=30)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  5.80it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [6]:
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>Tell me what did you see in this picture?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

A red bus, a Starbucks, and a car parked in front of it.<|eot_id|>
